# Algo de recommandation

In [30]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

In [31]:
def recommand_product(df, product_name, tsne_cols=['tsne1', 'tsne2', 'tsne3'], cluster_col='cluster'):
    """
    Recommande un produit le plus proche dans chaque cluster pour un produit donné.

    Paramètres :
        df (pd.DataFrame) : Le DataFrame contenant les colonnes t-SNE, cluster, et product_name.
        product_name (str) : Le nom du produit choisi.
        tsne_cols (list) : Colonnes représentant les coordonnées dans l'espace vectoriel.
        cluster_col (str) : Colonne représentant les clusters.

    Retourne :
        pd.DataFrame : Les produits recommandés (le plus proche dans chaque cluster).
    """
    # Vérifier si le produit existe dans le DataFrame
    if product_name not in df['product_name'].values:
        raise ValueError(f"Le produit {product_name} n'existe pas dans le DataFrame.")
    
    # Extraire les informations du produit choisi
    chosen_product = df[df['product_name'] == product_name]
    chosen_cluster = chosen_product[cluster_col].values[0]
    chosen_coords = chosen_product[tsne_cols].values[0]

    # Initialiser une liste pour les recommandations
    recommendations = []

    # Parcourir les clusters
    for cluster in df[cluster_col].unique():
        
        # Produits appartenant au cluster actuel
        cluster_products = df[df[cluster_col] == cluster]
                
        if cluster == chosen_cluster:
            # Dans le même cluster, exclure le produit choisi
            cluster_products = cluster_products[cluster_products['product_name'] != product_name]
        
        # Calculer la distance entre le produit choisi et les produits de ce cluster
        distances = cdist([chosen_coords], cluster_products[tsne_cols].values, metric='euclidean')
        
        # Trouver l'index du produit le plus proche
        closest_index = np.argmin(distances)
        
        # Ajouter le produit le plus proche à la liste des recommandations
        recommendations.append(cluster_products.iloc[closest_index])
    
    # Convertir les recommandations en DataFrame
    recommendations_df = pd.DataFrame(recommendations)
    return recommendations_df

# Test sur le dataset

In [32]:
df = pd.read_csv('./amazon_clusters.csv')
produit = df.sample(1).values[0]
nom_produit, cluster_produit = produit[2], produit[0]
print(f"Produit choisi : {nom_produit} (cluster {cluster_produit})")

Produit choisi : AGARO Royal Stand 1000W Mixer with 5L SS Bowl and 8 Speed Setting, Includes Whisking Cone, Mixing Beater & Dough Hook, and Splash Guard, 2 Years Warranty, (Black), Medium (33554) (cluster 4)


In [33]:
df_recommdations = recommand_product(df, nom_produit)
df_recommdations

,cluster,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,...,review_title,review_content,img_link,product_link,stock,embedding,text,tsne1,tsne2,tsne3
521,2,B0B8ZWNR5T,STRIFF 12 Pieces Highly Flexible Silicone Micr...,Electronics|Mobiles&Accessories|MobileAccessor...,₹79,₹499,84%,4.2,"1,949",AUDIO BOOST which rewards your ears with studi...,...,"Very useful,Very useful item to make your phon...",A 10bucks piece can save cable worth hundreds....,https://m.media-amazon.com/images/I/41R0DrIbTN...,https://www.amazon.in/STRIFF-Flexible-Silicone...,80,"[-0.048309326171875, 0.0020599365234375, 0.030...",product_name striff 12 piece highly flexible s...,5.846655,7.387490,1.305743
997,5,B00URH5E34,Inventis 5V 1.2W Portable Flexible USB LED Lig...,Computers&Accessories|Accessories&Peripherals|...,₹39,₹39,0%,3.6,"13,572",The side the LED light falls on will depend on...,...,"it worked properly for almost one year,ok,USB ...","very good product, very bright, easy to use, i...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Inventis-Portable-Flexib...,116,"[-0.0416259765625, 0.022369384765625, 0.010704...",product_name inventis 5v 12w portable flexible...,0.269788,5.101322,3.033293
242,0,B0758F7KK7,Caprigo Heavy Duty TV Wall Mount Bracket for 1...,"Electronics|HomeTheater,TV&Video|Accessories|T...",₹399,₹999,60%,4.0,"1,236",TV Wall Bracket Compatibility : 14 To 32 Inch ...,...,Good quality product with many screws and nuts...,"My 32 inch tv fits fine, also 55 inch TV can b...",https://m.media-amazon.com/images/I/41js3ITzVH...,https://www.amazon.in/Maxicom-B-28-Universal-B...,113,"[-0.0162200927734375, -0.01549530029296875, 0....",product_name caprigo heavy duty tv wall mount ...,-5.411987,-6.043121,-8.195428
787,1,B07DKZCZ89,"Gizga Essentials Earphone Carrying Case, Multi...","Electronics|Headphones,Earbuds&Accessories|Cases",₹119,₹499,76%,4.3,"15,032",SUPERIOR QUALITY: Gizga Essentials Earphone Ca...,...,"Good quality,Good quality product,Must Go For ...","Good quality product at this price,Its good bu...",https://m.media-amazon.com/images/I/41+d7HRWPw...,https://www.amazon.in/Essentials-G11-Earphone-...,101,"[-0.05340576171875, -0.0110015869140625, 0.037...",product_name gizga essential earphone carrying...,5.350484,2.001190,2.860548
1104,3,B09VKWGZD7,"AGARO Supreme High Pressure Washer, 1800 Watts...","Home&Kitchen|Kitchen&HomeAppliances|Vacuum,Cle...","₹4,789","₹8,990",47%,4.3,"1,017","Includes 3 meters inlet pipe, 8 meters outlet ...",...,"good machine in budget,Very good product...,Ov...",Pros:1. Small size of the machine.2. Good mach...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Supreme-Pressure-Portabl...,111,"[-0.031951904296875, 0.006977081298828125, 0.0...",product_name agaro supreme high pressure washe...,-10.146971,13.547079,-5.834807
1336,4,B07R679HTT,AGARO Imperial 240-Watt Slow Juicer with Cold ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,"₹12,609","₹23,999",47%,4.4,"2,288",Cold press process for maximum juice extractio...,...,"Overall Nice Product,Must buy product,Product ...","It's a nice product, does the needful. Was eas...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/AGARO-Imperial-Slow-Juic...,92,"[-0.0146636962890625, 0.01806640625, 0.0227050...",product_name agaro imperial 240watt slow juice...,-9.874347,13.877889,-3.325840
